# Input catalogs

Various classes of astronomical objects of interests are typically shared as catalogs of targets, such as star clusters, Star Forming Regions etc.  This notebook develops IO functions to handle the different formats. 

In [11]:
import csv
from os import path
from astropy.table import Table, Column
from astropy.io import fits
import numpy as np 
import healpy as hp

# Configure path to local repository
root_dir = '/Users/rstreet/software/rgps'

The first input catalog received is a set of Star Forming Regions in CSV format, courtesy of Matthew De Furio. 

In [2]:
def load_catalog(root_dir, catalog_name):
    """
    Function to load a catalog of regions, defined as centroid locations plus a radial extent from that
    centroid, which is assumed to be circular.  Centroids can be defined in RA, Dec or (l,b), but
    all quantities should be in decimal degrees.  Since the catalogs have been shared in CSV format
    from different authors, this function handles the formatting specific to each catalog.

    :param root_dir: Path to the config directory where the catalog file can be found
    :param catalog_name: Name of the catalog file
    :return:
    pointing_set: List of dictionaries in the form {"pointing": [l_center, b_center, radius]} in decimal degrees
    """

    catalog_file = path.join(root_dir, catalog_name)
    if not path.isfile(catalog_file):
        raise IOError('Cannot find requested catalog ' + catalog_file)

    pointing_set = []

    if catalog_name == 'SFRs_for_Roman.csv':

        with open(catalog_file) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for i,row in enumerate(csv_reader):
                if i == 0:
                    header = row
                else:
                    pointing_set.append({"pointing": [float(row[4]), float(row[5]), float(row[7])]})

    return pointing_set

In [3]:
pointings = load_catalog(path.join(root_dir, 'config'), 'SFRs_for_Roman.csv')
pointings

[{'pointing': [264.5, 1.7, 0.6]},
 {'pointing': [78.5, 2.7, 0.6]},
 {'pointing': [356.5, -0.7, 0.6]},
 {'pointing': [85.2, -0.9, 0.7]},
 {'pointing': [266.5, -3.6, 0.9]},
 {'pointing': [78.9, 0.7, 0.7]},
 {'pointing': [245.2, -3.6, 1.0]},
 {'pointing': [261.4, 1.0, 0.7]},
 {'pointing': [280.7, 4.3, 0.8]},
 {'pointing': [181.4, -2.6, 0.7]},
 {'pointing': [341.3, -2.9, 1.1]},
 {'pointing': [57.6, -1.3, 1.2]},
 {'pointing': [57.3, 1.3, 1.0]},
 {'pointing': [33.8, 1.5, 1.2]},
 {'pointing': [195.5, -2.8, 1.1]},
 {'pointing': [260.0, 2.8, 1.6]},
 {'pointing': [202.8, 1.9, 1.4]},
 {'pointing': [269.2, 0.0, 1.5]},
 {'pointing': [91.0, 2.7, 1.4]},
 {'pointing': [240.0, -2.4, 1.6]},
 {'pointing': [256.2, -3.4, 1.5]},
 {'pointing': [255.9, -2.3, 2.0]},
 {'pointing': [119.1, 3.4, 1.8]},
 {'pointing': [104.2, -3.9, 1.2]},
 {'pointing': [248.4, -0.5, 1.5]},
 {'pointing': [56.1, 3.2, 1.8]},
 {'pointing': [70.5, -0.2, 2.5]},
 {'pointing': [85.3, -0.7, 2.0]},
 {'pointing': [127.3, 1.8, 2.9]},
 {'pointi

Next, we have a catalog of AGNs previously observed by Fermi-LAT obtained from their [Data Release 3](https://fermi.gsfc.nasa.gov/ssc/data/access/lat/4LACDR3/) on the recommendation of Filippo D'Ammando.

In [4]:
def load_AGN_catalog(root_dir, catalog_name):
    """
    Function to load a source catalog from Fermi-LAT DR3.
    """
    catalog_file = path.join(root_dir, catalog_name)
    if not path.isfile(catalog_file):
        raise IOError('Cannot find requested catalog ' + catalog_file)

    pointing_set = []
    
    with fits.open(catalog_file) as hdul:
        data = hdul[1].data
        for row in data:
            pointing_set.append({"pointing": [float(row[4]), float(row[5]), 0.3]})
    
    return pointing_set

In [5]:
pointing_set = load_AGN_catalog(path.join(root_dir, 'config'), 'table-4LAC-DR3-l.fits')
print(pointing_set)

[{'pointing': [119.66110229492188, 8.111637115478516, 0.3]}, {'pointing': [118.07474517822266, -4.498106479644775, 0.3]}, {'pointing': [118.55729675292969, -1.2571890354156494, 0.3]}, {'pointing': [117.88809967041016, -6.657064437866211, 0.3]}, {'pointing': [120.17926788330078, 6.626677513122559, 0.3]}, {'pointing': [121.07728576660156, -1.2922883033752441, 0.3]}, {'pointing': [120.98204803466797, -2.9796204566955566, 0.3]}, {'pointing': [121.7068099975586, -1.9695024490356445, 0.3]}, {'pointing': [122.31592559814453, 5.174300193786621, 0.3]}, {'pointing': [122.32572174072266, -5.906441688537598, 0.3]}, {'pointing': [122.42411804199219, -8.066608428955078, 0.3]}, {'pointing': [124.32300567626953, -4.064716815948486, 0.3]}, {'pointing': [124.4244613647461, -4.429605960845947, 0.3]}, {'pointing': [124.74527740478516, -9.203022003173828, 0.3]}, {'pointing': [125.11431121826172, -1.2325012683868408, 0.3]}, {'pointing': [124.68570709228516, 5.289400577545166, 0.3]}, {'pointing': [125.381820

There is a catalog of Globular Clusters, in a different CSV format...

In [14]:
catalog_file = path.join(root_dir, 'config', 'baumgardt_harris_GCs.csv')

pointing_set = []

pixscale = hp.max_pixrad(64,degrees=True)

with open(catalog_file, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', quotechar='|')
    for i,row in enumerate(csv_reader):
        if i >= 1:
            entries = row[0].split(',')
            print(entries)
            
            # The cluster radius is set to 1.5*half-light radius
            # The half-light radius is r_hl(pc) / R_sun(kpc) converted to deg
            # However, the radius needs to have a minimum of at least one HEALpixel to register
            # on the map
            try:
                radius = max((1.5 * (0.001 * float(entries[16]) / float(entries[5])) * 2.0 * (180.0 / np.pi)),
                         pixscale)
                pointing_set.append({"pointing": [float(entries[3]), float(entries[4]), radius]})

            # Skip malformed catalog entries
            except ValueError:
                pass

['NGC_104', '6.02379', '-72.08131', '305.895', '-44.889', '4.52', '0.03', '7.52', '0.01', '895000.0', '5850.0', '4.08', '0.06', '1.96', '0.09', '0.79', '3.64', '6.3', '126.8', '4.37', '2.65', '-17.45', '0.16', '11.9', '5.252', '0.021', '-2.551', '0.021', '-0.0', '4.63', '3.65', '9.73', '-0.83', '0.44', '47.5', '-0.37', '-0.04', '5.0', '0.32', '100.0', '6.3', '0.01', '-2.59', '0.02', '-3.19', '0.02', '77.52', '0.42', '170.92', '0.39', '45.95', '0.34', '5.47', '0.02', '7.51', '0.0', '-0.72', '0.04', '14.06', '3.95', '-9.42', '0.37', '0.88', '0.53', '1.14', 'G4', '0.09', '-18.0', '0.1', '-26.7', '11.0', '0.3', '2.07', '', '0.36', '3.17']
['NGC_288', '13.1885', '-26.58261', '151.285', '-89.38', '8.99', '0.09', '12.21', '0.06', '93400.0', '2580.0', '8.1', '0.06', '2.16', '0.1', '3.59', '5.83', '8.37', '94.86', '1.8', '1.28', '-44.45', '0.13', '3.1', '4.164', '0.024', '-5.705', '0.024', '0.01', '2.76', '2.43', '9.47', '-0.52', '0.49', '10.8', '-0.22', '-0.09', '0.42', '0.32', '84.3', '8.26',

An alternative version of the Baumgardt catalog was provided by Dante Minniti, in a slightly different format...

In [ ]:
catalog_file = path.join(root_dir, 'config', 'baumgardt_orbits_table_VVVX_all.txt')

pointing_set = []

pixscale = hp.max_pixrad(64,degrees=True)

with open(catalog_file) as txt_file:
    file_lines = txt_file.readlines()

    for line in file_lines:
        if line[0:1] != '#':
            entries = line.replace('\n', '').split()

            radius = max((1.5 * (0.001 * float(entries[16]) / float(entries[5])) * 2.0 * (180.0 / np.pi)),
                         pixscale)
            pointing_set.append({"pointing": [float(entries[4]), float(entries[5]), radius]})
